In [50]:
from getpass import getpass
import os

In [51]:
OPENAI_API_KEY = getpass('Ingresa tu APIKEY: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

Ingresa tu APIKEY:  ········


In [52]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://unl.edu.ec/sites/default/files/archivo/2023-09/Guía%20para%20la%20elaboración%20de%20proyectos%20de%20investigación%20para%20titulación.pdf',
    'https://unl.edu.ec/sites/default/files/archivo/2023-09/Guia%20Presentacion%20Informe%20Integración%20Curricular%20o%20Titulación.pdf',
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'documento{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargando {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

print('Contenido de ml_papers: ')
print()

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)


Descargando documento1.pdf


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 94 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)


Descargando documento2.pdf
Contenido de ml_papers: 



In [53]:
type(ml_papers), len(ml_papers), ml_papers[3]

(list,
 48,
 Document(page_content=' \n4       Presentación  La formación de investigadores en las instituciones de Educación Superior es fundamental para el desarrollo del conocimiento y su aplicación en acciones para el mejoramiento de las condiciones de vida de la población. La Universidad Nacional de Loja, consciente de esta realidad y en cumplimiento de sus funciones sustantivas: Docencia, Investigación y Vinculación con la Sociedad, impulsa a que los jóvenes universitarios se inicien y dispongan de los elementos técnicos y conceptos que complementen su formación y faciliten su involucramiento en acciones de investigación según su vocación. La investigación de integración curricular o de titulación propende al desarrollo de conocimientos y destrezas investigativas orientadas a la innovación científica, tecnológica, social, humanística y artística; busca crear una cultura de investigación desde los estudiantes; por lo tanto, se requiere que el proceso sea riguroso y se inicie media

In [54]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [55]:
len(documents), documents[10]

(75,
 Document(page_content='8  Deben ser coherentes con las preguntas específicas de la investigación, estar formulados con verbos en infinitivo. Se sugiere utilizar los verbos de acción de la taxonomía de Bloom (Aguirre, 2014).  5. Marco teórico Para construir el marco teórico es necesario que la información que se reúna y analice, se relacione de manera coherente entre sí y con el objeto de estudio, evitando divagar con el planteamiento de temas no vinculados directamente al objeto de investigación (Hernández-Sampieri, 2018). Se incluirá toda aquella información ya documentada que se tiene sobre el tema de estudio, realizando una sistematización del conocimiento científico en relación a la temática a estudiar, que evidencie el avance de la ciencia y los vacíos de conocimientos que orientan la investigación (Dirección de Investigación-UNL, 2021). Toda la información bibliográfica usada debe estar respaldada por sus respectivas citas, las cuales deben ser incluidas en la listas de ref

In [56]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002") 

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
    )

retriever = vectorstore.as_retriever(
    search_kwargs={"k" : 3}
    )

In [57]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [58]:
query = "Para que sirve la GUÍA PARA LA ESCRITURA Y PRESENTACIÓN DEL INFORME DEL TRABAJO DE INTEGRACIÓN CURRICULAR O DE TITULACIÓN"
qa_chain.invoke(query)

{'query': 'Para que sirve la GUÍA PARA LA ESCRITURA Y PRESENTACIÓN DEL INFORME DEL TRABAJO DE INTEGRACIÓN CURRICULAR O DE TITULACIÓN',
 'result': 'La GUÍA PARA LA ESCRITURA Y PRESENTACIÓN DEL INFORME DEL TRABAJO DE INTEGRACIÓN CURRICULAR O DE TITULACIÓN sirve para apoyar a la comunidad universitaria en la uniformización de la presentación de los resultados de sus trabajos de integración curricular o de titulación. Su propósito es ayudar a los estudiantes y tutores a escribir informes de manera más efectiva, mejorar sus habilidades para comunicar resultados científicos, y preparar a los estudiantes para la redacción y presentación de informes técnicos y profesionales en el ejercicio de sus labores. Además, busca apoyar la preparación de los estudiantes para la realización de publicaciones científicas y para sus estudios de postgrado.'}

In [59]:
query = "Que va en los resultados?"
qa_chain.run(query)

'En la sección de resultados de una investigación se exponen y describen los datos obtenidos, se interpretan y contrastan con la teoría, el estado de la cuestión y la propia investigación. Los resultados teóricos enriquecen, modifican o perfeccionan la teoría científica con nuevos conocimientos sobre el objeto y los métodos de la investigación. Los resultados deben ser claros, precisos y escritos en pretérito o participio pasado, en una secuencia lógica en el texto. Se enfatizan solo las observaciones importantes y se pueden utilizar recursos como tablas y figuras para presentar la información de manera organizada. Los resultados deben seguir el mismo orden en que fueron recogidos, colocándolos de acuerdo a los objetivos planteados y al diseño de la metodología. Las tablas deben ir numeradas, contener un título descriptivo en la parte superior y no llevar líneas innecesarias. Las notas referentes a la tabla se colocan al final de la misma.'